In [ ]:
import numpy as np
import pandas as pd
from transformers import pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import torch

from torch.utils.data import Dataset
import torch.nn.functional as F
from wordcloud import WordCloud, STOPWORDS
import warnings
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd

data=pd.read_csv('//content/cleaned_amazon_reviews.csv')
data.head()

,Sentiment,Title,Review,Cleaned_Review
0,0,Wanna be anti war movie,"[...].As a Vietnam veteran, one tour enlisted,...",vietnam veteran one tour enlisted one tour off...
1,1,Beyond the Wild Bunch,I found the book to be very informative and he...,found book informative helpful however title b...
2,0,Disapointed,"The movie was OK, certainly not as good as the...",movie certainly good others however biggest pr...
3,0,PUSS - stick to shrek,I was so looking forward to this film. I actua...,looking forward film actually waited thanksgiv...
4,0,"rude, bad grammar girl... not good for young k...",My son and I just read through our first book ...,son read first book disappointed rude lots hat...


In [ ]:
data.Sentiment.value_counts()

,count
Sentiment,
0,10069
1,9931


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Sentiment       20000 non-null  int64 
 1   Title           19998 non-null  object
 2   Review          20000 non-null  object
 3   Cleaned_Review  20000 non-null  object
dtypes: int64(1), object(3)
memory usage: 625.1+ KB


In [ ]:
data=data.drop(['Title','Review'],axis=1)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Sentiment       20000 non-null  int64 
 1   Cleaned_Review  20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


In [ ]:
data.head()

,Sentiment,Cleaned_Review
0,0,vietnam veteran one tour enlisted one tour off...
1,1,found book informative helpful however title b...
2,0,movie certainly good others however biggest pr...
3,0,looking forward film actually waited thanksgiv...
4,0,son read first book disappointed rude lots hat...


In [ ]:
data.Cleaned_Review[0]

'vietnam veteran one tour enlisted one tour officer film depicts struggles complications conduct vietnam war well films depicted cluster last stand words even close directed someone never starred actors never results like never war duh know war hell many said many times many long years ago slap face fought well tried best bunch people unwilling place hides jeopardy philosophy tenuous freedom really bad matter good special effects expensive number extras'

In [ ]:
data.shape

(20000, 2)

In [ ]:
data,test_dataset=train_test_split(data,test_size=0.01,random_state=42,shuffle=True)

In [ ]:
data.shape,test_dataset.shape

((19800, 2), (200, 2))

In [ ]:
x=data['Cleaned_Review'].tolist()
id2label={0:'negative',1:'positive'}
label2id={'negative':0,'positive':1}
y=data['Sentiment'].tolist()

In [ ]:
class CustomDataset(Dataset):
  def __init__(self,texts,labels,tokeniser,max_len=64):
    self.texts=texts
    self.labels=labels
    self.tokeniser=tokeniser
    self.max_len=max_len
  def __len__(self):
    return len(self.texts)
  def __getitem__(self,idx):
    text=str(self.texts[idx])
    label=torch.tensor(self.labels[idx])
    encoding=self.tokeniser(text,truncation=True,padding='max_length',max_length=self.max_len)
    return {'input_ids':encoding['input_ids'],'attention_mask':encoding['attention_mask'],'labels':label}



In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
checkpoint='distilbert-base-uncased'
tokeniser=AutoTokenizer.from_pretrained(checkpoint)
model= AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset=CustomDataset(x,y,tokeniser)

In [ ]:
dataset[0].keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
train_dataset,val_dataset=train_test_split(dataset,test_size=0.1,random_state=42,shuffle=True)

In [ ]:
len(train_dataset),len(val_dataset)

(17820, 1980)

In [ ]:
from sklearn.metrics import accuracy_score,f1_score
def compute_metrics(pred):
  labels=pred.label_ids

  preds=pred.predictions.argmax(-1)
  f1=f1_score(labels,preds,average='weighted')
  acc=accuracy_score(labels,preds)
  return {'accuracy':acc,'f1':f1}

In [ ]:

batch_size=24
#model_name='distilbert_finetuned_setiment'
from transformers import trainer,TrainingArguments

In [ ]:
#batch_size=16
#model_name='distilbert_finetuned_setiment'
#from transformers import trainer,TrainingArguments # This line is causing the problem

# Import the Trainer class correctly
from transformers import Trainer, TrainingArguments
from transformers.trainer_callback import EarlyStoppingCallback
training_args = TrainingArguments(
    output_dir="output",          # output directory
    evaluation_strategy="epoch",     # evaluate during training
    save_strategy="epoch",           # save model every epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=batch_size,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    num_train_epochs=5,
     weight_decay=0.01,                      # add weight decay for regularization
    load_best_model_at_end=True# number of training epochs
)

# Create Trainer
early_stopping = EarlyStoppingCallback(early_stopping_patience=3)
# Changed 'validation_dataset' to 'eval_dataset'
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokeniser,
    callbacks=[early_stopping]
    # Assuming you meant 'tokenizer' here instead of 'tokeniser'
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.421300,0.383858,0.836364,0.835669
2,0.301600,0.471734,0.827273,0.825256
3,0.169700,0.465020,0.843434,0.843258
4,0.141000,0.634907,0.851515,0.851510


TrainOutput(global_step=2972, training_loss=0.23551338935122998, metrics={'train_runtime': 444.8208, 'train_samples_per_second': 200.305, 'train_steps_per_second': 8.352, 'total_flos': 1180284522024960.0, 'train_loss': 0.23551338935122998, 'epoch': 4.0})

In [ ]:
trainer.save_model("output")

output = AutoModelForSequenceClassification.from_pretrained('/content/output')
tokenizer = AutoTokenizer.from_pretrained('/content/output')
text='this is  good product'
# Pass the tokenizer object to the pipeline function
pipe=pipeline('text-classification', model=output, tokenizer=tokenizer)
pipe(text)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_1', 'score': 0.9512278437614441}]

In [ ]:
test_dataset=test_dataset.Cleaned_Review.tolist()

In [ ]:
texts = test_dataset

results = []

for text in texts:
    input_ids = tokenizer.encode(text, return_tensors='pt')
    out = output(input_ids)
    preds = torch.nn.functional.softmax(out.logits, dim=-1)
    prob = torch.max(preds).item()
    ids = torch.argmax(preds).item()
    sentiment = id2label[ids]


    results.append((text, sentiment, prob))


for result in results:
    print(f"Text: {result[0]}, Sentiment: {result[0]}, Probability: {result[1]}")

Text: store concerned buttons hard push complicated set decided give product try sleep alarm clock price seemed alright ad feature liked curved sleek shape unit unlike many looked confusion hard accidentally reset time sleep happy face smiley built buzzer alarm pretty good sound insistent pleasant mornings hard enough looked forward testing speaker quality one ads instead relying store radio reception though speakers sounded alright store unfortunately disappointed get unit home discover ad player work straight box reading reviews think trying different product instead taking risk one, Sentiment: store concerned buttons hard push complicated set decided give product try sleep alarm clock price seemed alright ad feature liked curved sleek shape unit unlike many looked confusion hard accidentally reset time sleep happy face smiley built buzzer alarm pretty good sound insistent pleasant mornings hard enough looked forward testing speaker quality one ads instead relying store radio recepti

In [ ]:
pd.DataFrame(results)

,0,1,2
0,store concerned buttons hard push complicated ...,negative,0.824931
1,get hot takes forever straighten hair fine hai...,positive,0.937939
2,bought item looked way better plastic sprinkle...,negative,0.901584
3,great idea lots problems various toner error m...,negative,0.812845
4,book advanced study nature ritual expression c...,positive,0.977228
...,...,...,...
195,one word describe game coming gamecube x box n...,negative,0.939137
196,bought laptop sleeve found great sleeve decide...,negative,0.955525
197,got microdot mouse followed directions install...,negative,0.963742
198,thought would add star vote think worst roger ...,negative,0.968967


In [ ]:
tested_data=pd.DataFrame(results, columns=['reviews', 'sentiment', 'probability'])

In [ ]:
tested_data

,reviews,sentiment,probability
0,store concerned buttons hard push complicated ...,negative,0.824931
1,get hot takes forever straighten hair fine hai...,positive,0.937939
2,bought item looked way better plastic sprinkle...,negative,0.901584
3,great idea lots problems various toner error m...,negative,0.812845
4,book advanced study nature ritual expression c...,positive,0.977228
...,...,...,...
195,one word describe game coming gamecube x box n...,negative,0.939137
196,bought laptop sleeve found great sleeve decide...,negative,0.955525
197,got microdot mouse followed directions install...,negative,0.963742
198,thought would add star vote think worst roger ...,negative,0.968967


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
trainer.save_model("/content/gdrive/MyDrive/FineTunedModel/output")

In [ ]:
tested_data = pd.DataFrame(results, columns=['reviews', 'sentiment', 'probability']) #Ensure this line is executed and results contains the data you need
tested_data.to_csv('tested_data.csv', index=False)

In [ ]:
tested_data

,reviews,sentiment,probability
0,store concerned buttons hard push complicated ...,negative,0.824931
1,get hot takes forever straighten hair fine hai...,positive,0.937939
2,bought item looked way better plastic sprinkle...,negative,0.901584
3,great idea lots problems various toner error m...,negative,0.812845
4,book advanced study nature ritual expression c...,positive,0.977228
...,...,...,...
195,one word describe game coming gamecube x box n...,negative,0.939137
196,bought laptop sleeve found great sleeve decide...,negative,0.955525
197,got microdot mouse followed directions install...,negative,0.963742
198,thought would add star vote think worst roger ...,negative,0.968967


In [ ]:
tested_data